In [3]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [174]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch

%matplotlib inline

In [202]:
name_getter = lambda x: x != 3
names = pd.read_table(
    "basin_dataset_public_v1p2/basin_mean_forcing/daymet/01/01013500_lump_cida_forcing_leap.txt",
    skiprows=name_getter,
    sep=" ",
).columns.to_numpy()
header = np.zeros(len(names) - 3, dtype=object)
header[0] = "Time"
header[1:] = names[4:]
del names
df = pd.read_table(
    "basin_dataset_public_v1p2/basin_mean_forcing/daymet/01/01013500_lump_cida_forcing_leap.txt",
    names=header,
    skiprows=4,
)
df["Time"] = pd.to_datetime(df["Time"])
df["Time"] = (df["Time"] - df["Time"][0]).dt.days

In [203]:
streamflow = pd.read_table(
    "basin_dataset_public_v1p2/usgs_streamflow/01/01013500_streamflow_qc.txt",
    header=None,
    sep="\s+",
).astype("str")
del streamflow[0]
for i in range(len(streamflow[1])):
    streamflow[1][i] += " " + streamflow[2][i] + " " + streamflow[3][i]
streamflow[1] = pd.to_datetime(streamflow[1])
del streamflow[2]
del streamflow[3]
streamflow[1] = (streamflow[1] - streamflow[1][0]).dt.days
df = df[streamflow[5] != "M"]
streamflow = streamflow[streamflow[5] != "M"]
del streamflow[5]
streamflow.columns = ["Time", "Flow"]
streamflow
# test = streamflow[1][1] - streamflow[1][0]
# plt.scatter(test.days, 1)

,Time,Flow
0,0,655.0
1,1,640.0
2,2,625.0
3,3,620.0
4,4,605.0
...,...,...
12687,12687,207.0
12688,12688,200.0
12689,12689,196.0
12690,12690,189.0


In [198]:
device = "cuda:0"


class Net(nn.Module):
    def __init__(self, nodes=51, layers=1, inputs=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size=inputs, hidden_size=nodes, num_layers=layers).to(
            device
        )
        self.lstm = nn.DataParallel(
            self.lstm, device_ids=["cuda:0", "cuda:1"], output_device="cuda:0"
        )
        self.output_layer = nn.Linear(nodes, 1).to(device)

    def forward(self, x):
        x, (h_n, c_n) = self.lstm(x)
        return self.output_layer(x)

In [208]:
dayl = torch.from_numpy(df["dayl(s)"].to_numpy()).view(-1, 1).float().to(device)[:]
prcp = torch.from_numpy(df["prcp(mm/day)"].to_numpy()).view(-1, 1).float().to(device)[:]
y = (
    torch.from_numpy(streamflow["Flow"].to_numpy().astype("float"))
    .view(-1, 1)
    .float()
    .to(device)[:]
)

x = torch.zeros((len(df["Time"]), 1, 2)).to(device)
x[:, :, 0] = dayl[:]
x[:, :, 1] = prcp[:]

In [212]:
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=1)
loss_func = nn.MSELoss()
model.train()
for i in range(100):
    optimizer.zero_grad()
    y_pred = model.forward(x)
    loss = loss_func(y_pred, y)
    print(f"Epoch {i}: {loss.item()}")
    loss.backward()
    optimizer.step()

Epoch 0: 5611417.5
Epoch 1: 5581195.5
Epoch 2: 5551172.5
Epoch 3: 5521350.0
Epoch 4: 5491730.5
Epoch 5: 5462319.5
Epoch 6: 5433118.5
Epoch 7: 5404130.5
Epoch 8: 5375358.5
Epoch 9: 5346806.0
Epoch 10: 5318474.5
Epoch 11: 5290367.0
Epoch 12: 5262486.5
Epoch 13: 5234835.0
Epoch 14: 5207413.5
Epoch 15: 5180225.0
Epoch 16: 5153272.0
Epoch 17: 5126554.5
Epoch 18: 5100076.0
Epoch 19: 5073836.0
Epoch 20: 5047837.5
Epoch 21: 5022080.5
Epoch 22: 4996566.0
Epoch 23: 4971296.5
Epoch 24: 4946270.5
Epoch 25: 4921489.5
Epoch 26: 4896955.0
Epoch 27: 4872667.0
Epoch 28: 4848624.5
Epoch 29: 4824828.5
Epoch 30: 4801279.5
Epoch 31: 4777976.5
Epoch 32: 4754920.0
Epoch 33: 4732109.5
Epoch 34: 4709544.5
Epoch 35: 4687225.0
Epoch 36: 4665149.5
Epoch 37: 4643318.0
Epoch 38: 4621729.5
Epoch 39: 4600385.0
Epoch 40: 4579280.5
Epoch 41: 4558416.5
Epoch 42: 4537793.0
Epoch 43: 4517407.5
Epoch 44: 4497260.0
Epoch 45: 4477349.0
Epoch 46: 4457672.5
Epoch 47: 4438230.5
Epoch 48: 4419020.5
Epoch 49: 4400042.0
Epoch 50: 